# ChatBot project
The chatbot will be trained on the dataset which contains categories (intents), pattern and responses. We use a special recurrent neural network (LSTM) to classify which category the user’s message belongs to and then we will give a random response from the list of responses.

## 5 steps to create a chatbot
1. Import and load the data file
2. Preprocess data
3. Create training and testing data
4. Build the model
5. Predict the response

In [1]:
import tensorflow as tf

In [2]:
# Initialize necessary variables
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

In [3]:
# Load data

import json
data_file = open("intents.json").read()
intents = json.loads(data_file)

In [4]:
import nltk 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle

import numpy as np
import random

In [5]:
# Import module for tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hunte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Tokenizing data

for intent in intents["intents"]:
  for pattern in intent["patterns"]:
    # tokenize each word
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    #add documents in corpus
    documents.append((w, intent["tag"]))
    # add to classes list
    if intent ["tag"] not in classes:
      classes.append(intent["tag"])

In [7]:
# import module for lemmatize
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hunte\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hunte\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [9]:
# Lemmatize each word and remove duplicate words from the list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between pattenrs and intents
print(len(documents),"documents")
# classes = intents
print(len(classes), "classes", classes)
# words = allwords vocabulary
print(len(words), "unique lemmatizer words", words)

pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatizer words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


## Create training and testing data

In [10]:
# Create training data
training = []
# create an empty array for output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence 
for doc in documents:
  # inintiate bag of words
  bag = []
  # list of tokenized words for the pattern
  pattern_words = doc[0]
  # lemmatize each word - create base word, in attempt to represent related words
  pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
  # create bag of words array with 1, if word match found in current pattern
  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
  # output is a "0" for each tag and "1" for current tag (for each pattern)
  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1
  training.append([bag, output_row])
# shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\hunte\AppData\Local\Temp/ipykernel_16192/2328125741.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


## Build the model
3 layers:

1. Layer has 128 neurons,
2. Layer has 64 neurons
3. Output layer contains number of neurons equal to number of intents to predict output intent with softmax

In [11]:
# Create a model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), 
                        activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(len(train_y[0]), activation="softmax")
])

# Compile the model Stochastic gradient descent 
# with Nesterov accelerated gradient gives good results for this model
model.compile(loss="categorical_crossentropy",
              optimizer = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, 
                                                  momentum=0.9, nesterov=True),
              metrics = ["accuracy"])

# Fit the model
history = model.fit(np.array(train_x), 
                    np.array(train_y),
                    epochs=200,
                    batch_size=5,
                    verbose=1)
model.save("chatbot_model.h5", history)

print("model created")

Epoch 1/200


C:\Users\hunte\Desktop\Jupyter\projects\env\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


10/10 [==============================] - 0s 2ms/step - loss: 2.1744 - accuracy: 0.1915
Epoch 2/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1537 - accuracy: 0.1915
Epoch 3/200
10/10 [==============================] - 0s 2ms/step - loss: 2.0787 - accuracy: 0.2128
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9017 - accuracy: 0.2553
Epoch 5/200
10/10 [==============================] - 0s 2ms/step - loss: 1.7983 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 2ms/step - loss: 1.6208 - accuracy: 0.4468
Epoch 7/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5370 - accuracy: 0.5957
Epoch 8/200
10/10 [==============================] - 0s 3ms/step - loss: 1.5386 - accuracy: 0.5957
Epoch 9/200
10/10 [==============================] - 0s 3ms/step - loss: 1.3223 - accuracy: 0.7021
Epoch 10/200
10/10 [==============================] - 0s 3ms/step - loss: 1.2624 - accuracy: 0.6596
Epoch 11/200
10/10 [=

10/10 [==============================] - 0s 2ms/step - loss: 0.1245 - accuracy: 0.9362
Epoch 84/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0257 - accuracy: 1.0000
Epoch 85/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 86/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0624 - accuracy: 1.0000
Epoch 87/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0774 - accuracy: 0.9574
Epoch 88/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0763 - accuracy: 0.9787
Epoch 89/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0254 - accuracy: 1.0000
Epoch 90/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0525 - accuracy: 1.0000
Epoch 91/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 92/200
10/10 [==============================] - 0s 3ms/step - loss: 0.0516 - accuracy: 1.0000
Epoch 93/200


10/10 [==============================] - 0s 2ms/step - loss: 0.0146 - accuracy: 1.0000
Epoch 165/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0098 - accuracy: 1.0000
Epoch 166/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0438 - accuracy: 0.9787
Epoch 167/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 168/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0251 - accuracy: 1.0000
Epoch 169/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 170/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0230 - accuracy: 1.0000
Epoch 171/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0440 - accuracy: 1.0000
Epoch 172/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 173/200
10/10 [==============================] - 0s 2ms/step - loss: 0.0268 - accuracy: 1.0000
Epoc